# PowerShell Function Parameter Input Validation
So as to have the most reliable and predictable outcomes from our functions, we should validate parameter values supplied by consumers!

Through example function definitions here, let us explore some of at least the in-built ways in which PowerShell will validate the things for us!

## General Tips
Some general tips for validating parameter values in PowerShell functions:
- leverage `Allow*` and `Validate*` parameter attributes for parameters. More info in the PowerShell help [about_functions_advanced_parameters](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_functions_advanced_parameters#parameter-and-variable-validation-attributes)
- use expicit type declarations for your parameters for much in-built validation (is value already of type X? Then it already conforms to the needed/expected)
    - this also helps with `ValueFromPipeline` succinctness / simplification, for when it's time to be advanced in the "taking parameter values from pipeline" aspect

## Examples
Here we have a collection of various wireframe functions that illustrate the use of several of the parameter attributes for parameter validation, as well as using newly created enumerations. Also, we have an example of leveraging an existing object type. See the "functions advaned parameters" help above for the comprehensive docs.

### Using `ValidateLength`
Need to ensure the a given parameter's value is of a certain length? Sure!

In [8]:
function New-Thingy {
    <#  .Description
        Create a new thingy of the given name
    #>
    [CmdletBinding()]
    param(
        ## The name to use for the new thingy. Must be of acceptable length
        [ValidateLength(6,32)][String[]]$Name
    )
    process {$Name | Foreach-Object {Write-Verbose "Cool -- creating new thingy named '$_'"}}
}

Let's see -- these param values of valid length?

In [20]:
New-Thingy -Name shrty, longggggg -Verbose

New-Thingy: Cannot validate argument on parameter 'Name'. The character length (5) of the argument is too short. Specify an argument with a length that is greater than or equal to "6", and then try the command again.


Oh, no, a parameter value was too short! Ok, let's try again:

In [9]:
New-Thingy -Name lessShort, longggggg -Verbose

VERBOSE: Cool -- creating new thingy named 'lessShort'
VERBOSE: Cool -- creating new thingy named 'longggggg'


Success!
### Using `ValidateRange`
How about ensuring that a parameter's value is within a particular range? Fine!

In [18]:
function Set-Temperature {
    <#  .Description
        Set the thermostat to some temperature for maximum comfort
    #>
    [CmdletBinding()]
    param(
        ## To what temperature (Farenheit) this set the thermostat?
        [ValidateRange(55, 80)][Int]$Temperature
    )

    process {Write-Verbose "Oh, boy -- setting the temperature to '$Temperature'"}
}

Ok, how are these temperatures?

In [19]:
Set-Temperature -Temperature 71 -Verbose
Set-Temperature -Temperature 80 -Verbose

VERBOSE: Oh, boy -- setting the temperature to '71'
VERBOSE: Oh, boy -- setting the temperature to '80'


Noice -- that seemingly worked. How about ensuring that too cold or too hot are invalid values?

In [15]:
Set-Temperature -Verbose -Temperature 0

Set-Temperature: Cannot validate argument on parameter 'Temperature'. The 0 argument is less than the minimum allowed range of 55. Supply an argument that is greater than or equal to 55 and then try the command again.


In [16]:
Set-Temperature -Verbose -Temperature 212

Set-Temperature: Cannot validate argument on parameter 'Temperature'. The 212 argument is greater than the maximum allowed range of 80. Supply an argument that is less than or equal to 80 and then try the command again.


Seems legit -- we'll keep a nice, comfortable abode 😎
### Using `ValidateSet`
Oh, need to have a parameter value that is one of some predefined set of values? ¡No problemo!

In [22]:
function Start-PandoraStation {
    <#  .Description
        Start the given Pandora station
    #>
    [CmdletBinding()]
    param(
        ## The station to start. Pick a wiener!
        [ValidateSet("Slayer", "Metallica", "ABRB", "TBDM")][String]$Name = "Slayer"
    )

    process {Write-Verbose "Starting Pandora station '$Name Radio'"}
}

Ok, let's rock.

In [23]:
Start-PandoraStation -Verbose -Name ABRB

VERBOSE: Starting Pandora station 'ABRB Radio'


Yiss! 🤘

How about some easier listening -- does our function allow this kind of chill?

In [28]:
Start-PandoraStation -Verbose -Name AᗺBA

Start-PandoraStation: Cannot validate argument on parameter 'Name'. The argument "AᗺBA" does not belong to the set "Slayer,Metallica,ABRB,TBDM" specified by the ValidateSet attribute. Supply an argument that is in the set and then try the command again.


Mhmm -- the function author was having no '70s supergroup in here. No 'Dancing Queen' for you!
### Using PowerShell v5+'s `enum`s or `class`es
We can use an [enum](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_enum) (or, similarly, a [class](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_classes)) to define what is the type of the parameter value that is accepted and valid.

In [29]:
## define some enum for later use
enum AcceptableBand {
    Slayer
    Metallica
    ABRB
    TBDM
}

function Start-PandoraStation_UseParamType {
    <#  .Description
        Start the given Pandora station
    #>
    [CmdletBinding()]
    param(
        ## The station to start
        [AcceptableBand]$Name = "Slayer"
    )

    process {Write-Verbose "Starting Pandora station '$Name Radio'"}
}

Ok, can we still rock?

In [30]:
Start-PandoraStation -Verbose -Name TBDM

VERBOSE: Starting Pandora station 'TBDM Radio'


Still yiss! 🤘

And, does our function still prevent chill?

In [31]:
Start-PandoraStation -Verbose -Name AᗺBA

Start-PandoraStation: Cannot validate argument on parameter 'Name'. The argument "AᗺBA" does not belong to the set "Slayer,Metallica,ABRB,TBDM" specified by the ValidateSet attribute. Supply an argument that is in the set and then try the command again.


Noice -- same parameter validation, now with a reusable enumeration (say, for use in other functions / modules / etc.)!

### Using Existing Object Models
Similarly, we can use existing types to define what is the type of the parameter value that is accepted and valid. This can take care of much of the lifting (no need to define our own thingies, leverage well-known objects already in use by consumers!).

In [32]:
function Get-VMStuff {
    <#  .Description
        Get some pertinent info for the given VMware VM

        .Example
        Get-VM myVm | Get-VMStuff
    #>
    [CmdletBinding()]
    param(
        ## The VM(s) for which to get stuff
        [parameter(ValueFromPipeline = $true)][VMware.VimAutomation.Types.VirtualMachine[]]$VM
    )
    process {
        $VM | Foreach-Object {
            $_ | Select-Object -Property Name, NumCPU, @{Name = "NIC"; e={($script:ThisNic = $_ | Get-NetworkAdapter)}}, @{Name = "NetworkName"; e={$ThisNic.NetworkName}}, VMHost
        }
    }
}

K, let's try this out

In [34]:
Get-VM myCoolVM | Get-VMStuff


Name        : myCoolVM
NumCpu      : 1
NIC         : Network adapter 1
NetworkName : VMNet0.VLAN391
VMHost      : myhost0.dom.com




Alright, so now we can just pipe in some `VirtualMachine` object, and our function does all the goodness -- we didn't have to do any verification on the in put beyond specifying the object type we required! Poof -- the use of parameter types for parameter validation! Next step, incorporate such things when suitable into your hot functions!